This file will be the model in itself, in it we will split the data, pre-process it and then use a model to predict prices in the test data

# Pre-process

In [1648]:
import os
import warnings
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [1649]:
#The first step is that we have to split the data correctly
df_final = pd.read_csv('ready_to_learn_csv.csv')
y = df_final['Price'] 
X = df_final.drop(['Price'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2)#, random_state=0)#random state, if given it will be always the same split

In [1650]:
def ordinal_encoding():
    """
    change the categorical value with the ordinal encoding method
    """
    global X_train
    global X_test
    
    X_train = X_train.replace({"Furnished": {False: 0, True: 1}})
    X_train = X_train.replace({"Open_fire": {False: 0, True: 1}})
    X_train = X_train.replace({"Swimming_pool": {False: 0, True: 1}})
    X_train = X_train.replace({"State_of_the_building":{"AS_NEW": 6, "JUST_RENOVATED": 5, "GOOD": 4, "TO_BE_DONE_UP": 3, "TO_RENOVATE": 2, "TO_RESTORE": 1}})#check meaning of these
    X_test = X_test.replace({"Furnished": {False: 0, True: 1}})
    X_test = X_test.replace({"Open_fire": {False: 0, True: 1}})
    X_test = X_test.replace({"Swimming_pool": {False: 0, True: 1}})
    X_test = X_test.replace({"State_of_the_building":{"AS_NEW": 6, "JUST_RENOVATED": 5, "GOOD": 4, "TO_BE_DONE_UP": 3, "TO_RENOVATE": 2, "TO_RESTORE": 1}})

In [1651]:
def get_median_price_per_sm():
    """
    For each localities in the X_train part, this function will compute the median price per square meter, 
    then it will merge these values with X_train and X_test and will delete the collumn locality
    """
    global X_train
    global X_test

    #creation of a new dataframe, containing locality and median price per square meter of the locality (on X_train)
    area = X_train["Living_Area"]+ df_final["Terrace_Area"] + df_final["Garden_Area"]
    X_train["Price_Squared_Meter"] = df_final["Price"] / area
    median_price_meter = X_train.groupby("Locality")["Price_Squared_Meter"].median()
    del X_train["Price_Squared_Meter"]

    #merge this new df with x_train and x_test
    X_train = X_train.merge(median_price_meter, how='left', on='Locality')
    X_test = X_test.merge(median_price_meter, how='left', on='Locality')

    #delete Locality collumn
    del X_train['Locality']
    del X_test['Locality']

In [1652]:
def missing_values():
    """
    this function will deal with missing values, for the needed columns it adds a new column_wasmissing, 
    and 'replace' the missing values with the help of a simple imputer
    """
    global X_train
    global X_test
    cols_with_missing = ['Number_of_facades', 'Furnished', 'Swimming_pool', 'State_of_the_building', 'Price_Squared_Meter'] #type of properties, 'Number_of_rooms', 'Open_fire', Price_Squared_Meter
    for col in cols_with_missing:
        X_train[col + '_was_missing'] = X_train[col].isnull()+0.0000001
        X_test[col + '_was_missing'] = X_test[col].isnull()+0.0000001
    
    my_imputer = SimpleImputer()
    imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
    imputed_X_test = pd.DataFrame(my_imputer.fit_transform(X_test))
    # Imputation removed column names; put them back
    imputed_X_train.columns = X_train.columns 
    imputed_X_test.columns = X_test.columns 

    X_train = imputed_X_train
    X_test = imputed_X_test

In [1653]:
def standardize():
    """
    This function standardize each columns
    """
    global X_train
    global X_test
    for column in X_train.columns :
        # Get mean and standard deviation from training set (per feature)
        mean = np.mean(X_train[column])
        stdev = np.std(X_train[column])

        # Standardize training and testing set using the mean and standard deviation from the training set
        X_train[column] = (X_train[column] - mean) / stdev
        X_test[column] = (X_test[column] - mean) / stdev

In [1654]:
def preprocess():
    """
    this function will preprocess the data, it deals with nan values, non numerical values and normalize the columns
    """
    global X_train
    global X_test

    #deal with categorical variables----------------------------------------------------------------------------------------------------------------------------------------------------
    #one hot encoding
    X_train = pd.get_dummies(X_train, columns=["Type_of_property"], drop_first=True)
    X_test = pd.get_dummies(X_test, columns=["Type_of_property"], drop_first=True)

    #ordinal encoding
    ordinal_encoding()

    #locality is an int but we consider it as a categorical variabe, we will replace this column by the median price per square meter for each different localities
    get_median_price_per_sm()

    #deal with missing values---------------------------------------------------------------------------------------------------------------------------------------------------------------
    missing_values()

    #standardize----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    standardize()

    #resampling-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    #todo
    

In [1655]:
preprocess()
#X_test.to_csv('test.csv', index = False)

# Prediction

In [1656]:
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(random_state=0, n_estimators=300, max_depth=5)
model.fit(X_train, y_train)
preds = model.predict(X_test)
print("mean absolute error", mean_absolute_error(y_test, preds))

score = model.score(X_train, y_train)
print("Evaluating the model on the training set yields an accuracy of {}%".format(score*100))
score=model.score(X_test, y_test)
print("Evaluating the model on the testing set yields an accuracy of {:.2f}%".format(score*100))

mean absolute error 119596.98747422134
Evaluating the model on the training set yields an accuracy of 94.83505477253289%
Evaluating the model on the testing set yields an accuracy of 78.56%


In [1657]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=5, random_state=0)
model.fit(X_train, y_train)
preds = model.predict(X_test)
print("mean absolute error", mean_absolute_error(y_test, preds))

#r-squared error
score = model.score(X_train, y_train)
print("Evaluating the model on the training set yields an accuracy of {}%".format(score*100))
score=model.score(X_test, y_test)
print("Evaluating the model on the testing set yields an accuracy of {:.2f}%".format(score*100))

mean absolute error 131312.35442463966
Evaluating the model on the training set yields an accuracy of 93.32448474493188%
Evaluating the model on the testing set yields an accuracy of 73.74%


In [1658]:
"""
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor(n_neighbors=5)
model.fit(X_train, y_train)
preds = model.predict(X_test)
print("mean absolute error", mean_absolute_error(y_test, preds))

score = model.score(X_train, y_train)
print("Evaluating the model on the training set yields an accuracy of {}%".format(score*100))
score=model.score(X_test, y_test)
print("Evaluating the model on the testing set yields an accuracy of {:.2f}%".format(score*100))
"""


'\nfrom sklearn.neighbors import KNeighborsRegressor\n\nmodel = KNeighborsRegressor(n_neighbors=5)\nmodel.fit(X_train, y_train)\npreds = model.predict(X_test)\nprint("mean absolute error", mean_absolute_error(y_test, preds))\n\nscore = model.score(X_train, y_train)\nprint("Evaluating the model on the training set yields an accuracy of {}%".format(score*100))\nscore=model.score(X_test, y_test)\nprint("Evaluating the model on the testing set yields an accuracy of {:.2f}%".format(score*100))\n'

In [1659]:
"""
from sklearn.ensemble import AdaBoostRegressor

model = AdaBoostRegressor(random_state=0, n_estimators=3)
model.fit(X_train, y_train)
preds = model.predict(X_test)
print("mean absolute error", mean_absolute_error(y_test, preds))

score = model.score(X_train, y_train)
print("Evaluating the model on the training set yields an accuracy of {}%".format(score*100))
score=model.score(X_test, y_test)
print("Evaluating the model on the testing set yields an accuracy of {:.2f}%".format(score*100))
"""

'\nfrom sklearn.ensemble import AdaBoostRegressor\n\nmodel = AdaBoostRegressor(random_state=0, n_estimators=3)\nmodel.fit(X_train, y_train)\npreds = model.predict(X_test)\nprint("mean absolute error", mean_absolute_error(y_test, preds))\n\nscore = model.score(X_train, y_train)\nprint("Evaluating the model on the training set yields an accuracy of {}%".format(score*100))\nscore=model.score(X_test, y_test)\nprint("Evaluating the model on the testing set yields an accuracy of {:.2f}%".format(score*100))\n'

In [1660]:
"""
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(random_state=2)
model.fit(X_train, y_train)
preds = model.predict(X_test)
print("mean absolute error", mean_absolute_error(y_test, preds))

score = model.score(X_train, y_train)
print("Evaluating the model on the training set yields an accuracy of {}%".format(score*100))
score=model.score(X_test, y_test)
print("Evaluating the model on the testing set yields an accuracy of {:.2f}%".format(score*100))
"""

'\nfrom sklearn.tree import DecisionTreeRegressor\n\nmodel = DecisionTreeRegressor(random_state=2)\nmodel.fit(X_train, y_train)\npreds = model.predict(X_test)\nprint("mean absolute error", mean_absolute_error(y_test, preds))\n\nscore = model.score(X_train, y_train)\nprint("Evaluating the model on the training set yields an accuracy of {}%".format(score*100))\nscore=model.score(X_test, y_test)\nprint("Evaluating the model on the testing set yields an accuracy of {:.2f}%".format(score*100))\n'

In [1661]:
"""
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

model = LinearRegression()
model.fit(X_train_poly, y_train)

preds = model.predict(X_test_poly)
print("mean absolute error", mean_absolute_error(y_test, preds))

score = model.score(X_train_poly, y_train)
print("Evaluating the model on the training set yields an accuracy of {}%".format(score*100))
score=model.score(X_test_poly, y_test)
print("Evaluating the model on the testing set yields an accuracy of {:.2f}%".format(score*100))
"""

'\nfrom sklearn.preprocessing import PolynomialFeatures\nfrom sklearn.linear_model import LinearRegression\n\npoly = PolynomialFeatures(degree=2)\nX_train_poly = poly.fit_transform(X_train)\nX_test_poly = poly.transform(X_test)\n\nmodel = LinearRegression()\nmodel.fit(X_train_poly, y_train)\n\npreds = model.predict(X_test_poly)\nprint("mean absolute error", mean_absolute_error(y_test, preds))\n\nscore = model.score(X_train_poly, y_train)\nprint("Evaluating the model on the training set yields an accuracy of {}%".format(score*100))\nscore=model.score(X_test_poly, y_test)\nprint("Evaluating the model on the testing set yields an accuracy of {:.2f}%".format(score*100))\n'

Looking to the previous test we will investigate to optimise GradientBoostingRegressor and the random forest one (they are the 2 more promosing). We can see that the others get bad results, decisiontreeregressor seems to overfit